# Week 2: Vigenère Cipher

It's been a long time since I attacked a Vigenère cipher in earnest!
A good first step, as always, is to do a quick bit of research to see what kinds of attacks are available.

I based my solution on the [Index of Coincidence](https://en.wikipedia.org/wiki/Index_of_coincidence) method.
This allowed me to determine the length of the key very quickly using the Friedman test.

Once I had the key length, I split the ciphertext into columns. The first column contains all the characters that were encrypted with the first letter of the key, the second column contains all the characters that were encrypted with the second letter of the key, and so on. This allowed me to solve each column as a simple Caesar cipher!

The decrypted columns aren't going to be words, so we can't crib-drag them, but we can still do frequency analysis. This worked really well, even though my frequency analysis was a bit rough and ready. I *should* have used a proper statistcal test to compare the sample frequencies to the expected frequencies, but it was more than good enough for this challenge.

It's worth noting that my solution here is predicated on the plaintext being in English. If the plaintext was in a different language, I would have needed to use a different set of expected frequencies, and a different expected index of coincidence when determining the key length. Also, if the ciphertexts were very short, other methods would be more appropriate than a frequency analysis.

There are plenty of other approaches that I haven't used here. For example, you could run a quick dictionary attack on the key, trying likely English words to see if they decrypt the text into something sensible, or you could simply brute-force the shorter keys. You could also use other methods to determine the key length, such as the [Kasiski examination](https://en.wikipedia.org/wiki/Kasiski_examination).


In [75]:
# Before we get started with the challenges, let's set up some basics!
from typing import List

ALPHABET = "".join(chr(i + ord("A")) for i in range(26))
ENGLISH_INDEX_OF_COINCIDENCE = 1.73 / len(ALPHABET)


# It's a good idea to have a sanitizer function that removes non-alphabetic characters from the text and converts it to uppercase.
def strip_non_alpha(ciphertext: str) -> str:
    return "".join(char for char in ciphertext.upper() if char in ALPHABET)


# See https://en.wikipedia.org/wiki/Index_of_coincidence
def index_of_coincidence(text: str) -> float:
    text = strip_non_alpha(text)
    n = len(text)
    return sum(text.count(letter) * (text.count(letter) - 1) for letter in ALPHABET) / (
        n * (n - 1)
    )


# This function slices the text into n parts, where the i-th part contains every n-th character.
def slice_text(text: str, n: int) -> List[str]:
    text = strip_non_alpha(text)
    return [text[i::n] for i in range(n)]


# This function tries to find the period of the repeating key, i.e. the key length.
def find_key_length(ciphertext: str, max_key_length=20, tolerance=0.005) -> int:
    # Let's try key lengths from 1 to a sensible maximum
    for key_length in range(1, max_key_length + 1):
        # Group the ciphertext chars that were encrypted with the same key char
        parts = slice_text(ciphertext, key_length)
        # Calculate the index of coincidence for each group and take the mean
        mean_ioc = sum(index_of_coincidence(part) for part in parts) / key_length
        # If the average index of coincidence is close to the one of English text, we found the key length
        if abs(mean_ioc - ENGLISH_INDEX_OF_COINCIDENCE) < tolerance:
            return key_length
        
    # If we didn't find a likely key length, return -1
    return -1


In [76]:
import functools
import itertools



# We don't want to fetch the letter frequencies every time we call the function, so we cache the result.
@functools.lru_cache()
def english_letter_frequencies():
    import requests
    import json

    # Fetch and parse the letter frequencies from GitHub
    response = requests.get(
        "https://gist.githubusercontent.com/evilpacket/5973230/raw/045e0ecc34c6362728a9bce62d5cd2e41d29ad9a/letter_freq.json"
    )
    frequencies = json.loads(response.text)
    # Transform the keys to uppercase and normalize the values
    frequencies = {key.upper(): value / 100 for key, value in frequencies.items()}
    return frequencies


def english_correlation(text: str) -> float:
    text = strip_non_alpha(text)
    text = text.upper()
    n = len(text)
    english_frequencies = english_letter_frequencies()
    sample_frequencies = {letter: text.count(letter) / n for letter in ALPHABET}
    return sum(
        abs(english_frequencies[letter] - sample_frequencies[letter])
        for letter in ALPHABET
    )


def caesar_shift(text: str, shift: int) -> str:
    text = strip_non_alpha(text).upper()
    shift = shift % len(ALPHABET)
    shifted_alphabet = ALPHABET[shift:] + ALPHABET[:shift]
    shifted_text = "".join(
        shifted_alphabet[ALPHABET.index(letter)] if letter in ALPHABET else letter
        for letter in text
    )
    return shifted_text


def vigenere_decrypt(ciphertext: str, key: str) -> str:
    key = key.upper()
    plaintext = ""
    key_index = 0
    for char in ciphertext:
        if char in ALPHABET:
            shift = ALPHABET.index(key[key_index])
            plaintext += caesar_shift(char, -shift)
            key_index = (key_index + 1) % len(key)
        else:
            plaintext += char
    return plaintext


def crack_vigenere(ciphertext: str, max_key_length=20, correlation_threshold=0.5):
    # First, we need to find the key length
    key_length = find_key_length(ciphertext, max_key_length)
    print("Key length:", key_length)

    # Next, we need to slice the ciphertext into key_length columns
    columns = slice_text(ciphertext, key_length)

    # Each column was encrypted with the same key character, so we just need to solve each column as a Caesar cipher.
    # Now we need to find the most likely key character for each column
    # This is way more efficient than trying all possible keys!
    candidate_keys = []
    for column in columns:
        key_char_candidates = []
        for shift in range(len(ALPHABET)):
            # Shift the column and calculate the correlation with English letter frequencies
            shifted_column = caesar_shift(column, -shift)
            correlation = english_correlation(shifted_column)
            # If the correlation is close enough to 0, we found a likely key character!
            # We keep track of multiple candidates, as the correlation is not a perfect metric.
            if correlation < correlation_threshold:
                key_char_candidates.append(ALPHABET[shift])
        candidate_keys.append(key_char_candidates)

    # We now have a list of candidate key characters for each column.
    # Let's try all possible combinations of these candidates to find the correct key!
    # This works very well for these challenges with just a visual inspection of the output.
    # For a real-world scenario, you might automate this with a dictionary or a n-gram frequency analysis.
    for key in itertools.product(*candidate_keys):
        key = "".join(key)
        print("Candidate key:", key)
        plaintext = vigenere_decrypt(ciphertext, key)
        print("Plaintext:", plaintext.replace(" ", ""))

## Vigenère Challenge 1

The following ciphertext was produced by a Vigenère Cipher. Only the characters A-Z were encrypted; other characters were discarded and spaces included only for display purposes Luckily, your adversary was lax and used a short key of 5 characters or less! See if you can crack the code.

For all of these challenges, letting me know the source of the text is sufficient!



> VROMW GYIHW NWPOB HUOBO RNTEW DTIWT YIVAA PRKRL NHNHJ PYIJS NRCCB YWTRR JNPIJ NQIJS LYWAS JNCGI MMNRW EEWQI FWRSG IASJI AESMW EJSVE AARTU EWAVV XFQIL IQIMF NWQIB RQIFF AOGLN RVRUA JQAHH VBTZI AEYCU OBOND GLNGR RCLRQ JNOYC HVWOR VIWDZ VMAEG HSBSW DEIFT UIJTG IWTVS WOSXQ EESXM OCQIF JRNRX JLYTN RFSWH NRMSB QNFRE CUEIB NBFUE ZMNNN RMTUI AECSA TJLRC UAJSV RPEAI AAYGR RPYUA GMXNJ MCHVR OIIIV IAYCE FEOTR VQIFI WTEEW CRSOH VWQAI MWGGI WTUSD SNRMA LIJRG LNGRR CLRQN NCVXN BYWCR HQIZX XBREO IAIOI TYAEB JJMNR CHRPJ DVIBD RGUAE IMHRA JSZYL HUEWD FSVEE XQAAQ ABVRP LRCJN QLNWN WUOBO NDNXF IGLPR RECAQ QRRNX ROAJX RNFXU GLJLS XQERZ NNVRP TVPUH VWVAA RNRFK JVREM IFKDS GAQIP LCUER NDGLN TVHNO SLRSC SYUYE AIGCO OELNW NWMIF GXVRV NDGSK ECVXU QXXBR EKOII QIFGX MCEWY NRMAO SEEOI RNTTU ENWND NRMNB XJLYL RSYEA GRIBT NXNIA HNROC BHVVN CBYUD FEEEU MVFES VHNZR NTEVO FXOOE FRDQM WGQMB ATVNE NFUEP SDNGI WAAGN AAHKE VRPUA AXRGL HTBFN CBQYA EIMWV XQHVW ORVIW D




In [77]:
ciphertext = "VROMW GYIHW NWPOB HUOBO RNTEW DTIWT YIVAA PRKRL NHNHJ PYIJS NRCCB YWTRR JNPIJ NQIJS LYWAS JNCGI MMNRW EEWQI FWRSG IASJI AESMW EJSVE AARTU EWAVV XFQIL IQIMF NWQIB RQIFF AOGLN RVRUA JQAHH VBTZI AEYCU OBOND GLNGR RCLRQ JNOYC HVWOR VIWDZ VMAEG HSBSW DEIFT UIJTG IWTVS WOSXQ EESXM OCQIF JRNRX JLYTN RFSWH NRMSB QNFRE CUEIB NBFUE ZMNNN RMTUI AECSA TJLRC UAJSV RPEAI AAYGR RPYUA GMXNJ MCHVR OIIIV IAYCE FEOTR VQIFI WTEEW CRSOH VWQAI MWGGI WTUSD SNRMA LIJRG LNGRR CLRQN NCVXN BYWCR HQIZX XBREO IAIOI TYAEB JJMNR CHRPJ DVIBD RGUAE IMHRA JSZYL HUEWD FSVEE XQAAQ ABVRP LRCJN QLNWN WUOBO NDNXF IGLPR RECAQ QRRNX ROAJX RNFXU GLJLS XQERZ NNVRP TVPUH VWVAA RNRFK JVREM IFKDS GAQIP LCUER NDGLN TVHNO SLRSC SYUYE AIGCO OELNW NWMIF GXVRV NDGSK ECVXU QXXBR EKOII QIFGX MCEWY NRMAO SEEOI RNTTU ENWND NRMNB XJLYL RSYEA GRIBT NXNIA HNROC BHVVN CBYUD FEEEU MVFES VHNZR NTEVO FXOOE FRDQM WGQMB ATVNE NFUEP SDNGI WAAGN AAHKE VRPUA AXRGL HTBFN CBQYA EIMWV XQHVW ORVIW D"
crack_vigenere(ciphertext)

Key length: 4
Candidate key: JANE
Plaintext: MRBINGLEYWASGOODLOOKINGANDGENTLEMANLIKEHEHADAPLEASANTCOUNTENANCEANDEASYUNAFFECTEDMANNERSHISSISTERSWEREFINEWOMENWITHANAIROFDECIDEDFASHIONHISBROTHERINLAWMRHURSTMERELYLOOKEDTHEGENTLEMANBUTHISFRIENDMRDARCYSOONDREWTHEATTENTIONOFTHEROOMBYHISFINETALLPERSONHANDSOMEFEATURESNOBLEMIENANDTHEREPORTWHICHWASINGENERALCIRCULATIONWITHINFIVEMINUTESAFTERHISENTRANCEOFHISHAVINGTENTHOUSANDAYEARTHEGENTLEMENPRONOUNCEDHIMTOBEAFINEFIGUREOFAMANTHELADIESDECLAREDHEWASMUCHHANDSOMERTHANMRBINGLEYANDHEWASLOOKEDATWITHGREATADMIRATIONFORABOUTHALFTHEEVENINGTILLHISMANNERSGAVEADISGUSTWHICHTURNEDTHETIDEOFHISPOPULARITYFORHEWASDISCOVEREDTOBEPROUDTOBEABOVEHISCOMPANYANDABOVEBEINGPLEASEDANDNOTALLHISLARGEESTATEINDERBYSHIRECOULDSAVEHIMFROMHAVINGAMOSTFORBIDDINGDISAGREEABLECOUNTENANCEANDBEINGUNWORTHYTOBECOMPAREDWITHHISFRIEND


## Vigenère Challenge 2

Same story as above, but this time, the key's a bit longer. You might want to research good attacks to use against this cipher, rather than diving in with a brute force attack!



> ZOFGM WBMXD YTDEO WXIVO MDVFF HVEIS GQQAI LWFGZ QNZGM TRWZT YVMNG MDYKA ITGZU SNHZL QOMSS XQNTO MTTAM DXWQN CRINQ UXOJX TYOQU NKXTL VYQNT XAGEW MTVKB HNFYA ELINQ QQTRL UOELM LRLPI FGINK AITNK YEEUC SVWPT YXUSR DHEJT JOHLF HVBZV NJUOL LKOAU QRELB HRQIE IXACE MFIEB AEQSZ DJMCD VWPPV KPACK MLDHA TNKZA IKWWY QMSRF INJAF HRFQC EGECF IMMVY TTJVZ UGAZI JXBHR LDAEL QEALO RVTBU EWETY TBSJS DMRGL MHDFI GEGIA SPRFI WFJSF EIPQT UAZFZ GQTRU AMGEI CRFOY DXVWR FFTFT VDSJA OMXZT UAEGC HJENT AUKMP EVJXI KMTEN XRAZK ASRJQ NVBVT UWURR LAUES ZCVHN TUWUR VFXIE WAVVK UAGLQ RZMQS CGESZ UTEGZ MTKAM IAXGS FKQAH FPEIM PEZAO RFLKO CWPOK AMSNE QNFHV ETSHE RMPOH YTTKH BHRGX DVKEO EDPSF YAPNU QAJLW UEUQS FYPUZ SZDRG OEEGD TYHCG ULAFK AMMBF XYKHL IFEUS JMPEV VQAFY TISWG PFGBH REMSZ FXOFK UBCXW RVEBR FUIBY WUTZL KUEAA UJMWR RUMLC LWMRG RTYXU EALML YTJIG KAFKA WSRVQ PRKBE QVMYJ TBMBK FTVKZ EFLDI REUEA XMNTB MDGZQ RVFQG ULNEF MPEEE QNLIW NZSDS GXZHN HEIEY MRVGD TFMPE ZKQLM XAAAV DERWG TBOQL THUEN EUSJB WNNJK EEMMR CJUSV RMTNU DOJLB HRYGL WHNSC SOEDB VDFLT AKTZE GGAUI FQNQK MSFNZ SNJQT FMPOF WAFKA MBRSE TJMPA GHQRZ LPIAL QLCXK TFNMS KTVDP GALRG LUAKK MGTBH RLUCI XOAEV QDKAQ SRSDT YPQTU WZVZH CSRQQ SRGLS YGILP TVDFM DECRL RROFH VBZPY SZSRZ IIAKF UJTVD RSDLP BVTUW FWVGB IRLTC VGBUE QOADX BHRYD ERMLI FAXLL LQOAE QNK





In [78]:
ciphertext = "ZOFGM WBMXD YTDEO WXIVO MDVFF HVEIS GQQAI LWFGZ QNZGM TRWZT YVMNG MDYKA ITGZU SNHZL QOMSS XQNTO MTTAM DXWQN CRINQ UXOJX TYOQU NKXTL VYQNT XAGEW MTVKB HNFYA ELINQ QQTRL UOELM LRLPI FGINK AITNK YEEUC SVWPT YXUSR DHEJT JOHLF HVBZV NJUOL LKOAU QRELB HRQIE IXACE MFIEB AEQSZ DJMCD VWPPV KPACK MLDHA TNKZA IKWWY QMSRF INJAF HRFQC EGECF IMMVY TTJVZ UGAZI JXBHR LDAEL QEALO RVTBU EWETY TBSJS DMRGL MHDFI GEGIA SPRFI WFJSF EIPQT UAZFZ GQTRU AMGEI CRFOY DXVWR FFTFT VDSJA OMXZT UAEGC HJENT AUKMP EVJXI KMTEN XRAZK ASRJQ NVBVT UWURR LAUES ZCVHN TUWUR VFXIE WAVVK UAGLQ RZMQS CGESZ UTEGZ MTKAM IAXGS FKQAH FPEIM PEZAO RFLKO CWPOK AMSNE QNFHV ETSHE RMPOH YTTKH BHRGX DVKEO EDPSF YAPNU QAJLW UEUQS FYPUZ SZDRG OEEGD TYHCG ULAFK AMMBF XYKHL IFEUS JMPEV VQAFY TISWG PFGBH REMSZ FXOFK UBCXW RVEBR FUIBY WUTZL KUEAA UJMWR RUMLC LWMRG RTYXU EALML YTJIG KAFKA WSRVQ PRKBE QVMYJ TBMBK FTVKZ EFLDI REUEA XMNTB MDGZQ RVFQG ULNEF MPEEE QNLIW NZSDS GXZHN HEIEY MRVGD TFMPE ZKQLM XAAAV DERWG TBOQL THUEN EUSJB WNNJK EEMMR CJUSV RMTNU DOJLB HRYGL WHNSC SOEDB VDFLT AKTZE GGAUI FQNQK MSFNZ SNJQT FMPOF WAFKA MBRSE TJMPA GHQRZ LPIAL QLCXK TFNMS KTVDP GALRG LUAKK MGTBH RLUCI XOAEV QDKAQ SRSDT YPQTU WZVZH CSRQQ SRGLS YGILP TVDFM DECRL RROFH VBZPY SZSRZ IIAKF UJTVD RSDLP BVTUW FWVGB IRLTC VGBUE QOADX BHRYD ERMLI FAXLL LQOAE QNK"
crack_vigenere(ciphertext)

Key length: 8
Candidate key: MARTIANS
Plaintext: NOONEWOULDHAVEBELIEVEDINTHELASTYEARSOFTHENINETEENTHCENTURYTHATTHISWORLDWASBEINGWATCHEDKEENLYANDCLOSELYBYINTELLIGENCESGREATERTHANMANSANDYETASMORTALASHISOWNTHATASMENBUSIEDTHEMSELVESABOUTTHEIRVARIOUSCONCERNSTHEYWERESCRUTINISEDANDSTUDIEDPERHAPSALMOSTASNARROWLYASAMANWITHAMICROSCOPEMIGHTSCRUTINISETHETRANSIENTCREATURESTHATSWARMANDMULTIPLYINADROPOFWATERWITHINFINITECOMPLACENCYMENWENTTOANDFROOVERTHISGLOBEABOUTTHEIRLITTLEAFFAIRSSERENEINTHEIRASSURANCEOFTHEIREMPIREOVERMATTERITISPOSSIBLETHATTHEINFUSORIAUNDERTHEMICROSCOPEDOTHESAMENOONEGAVEATHOUGHTTOTHEOLDERWORLDSOFSPACEASSOURCESOFHUMANDANGERORTHOUGHTOFTHEMONLYTODISMISSTHEIDEAOFLIFEUPONTHEMASIMPOSSIBLEORIMPROBABLEITISCURIOUSTORECALLSOMEOFTHEMENTALHABITSOFTHOSEDEPARTEDDAYSATMOSTTERRESTRIALMENFANCIEDTHEREMIGHTBEOTHERMENUPONMARSPERHAPSINFERIORTOTHEMSELVESANDREADYTOWELCOMEAMISSIONARYENTERPRISEYETACROSSTHEGULFOFSPACEMINDSTHATARETOOURMINDSASOURSARETOTHOSEOFTHEBEASTSTHATPERISHINTELLECTSVASTANDCOOLANDUNSYMPATHE

## Vigenère Challenge 3

You know the drill; longer key, harder challenge. Best of luck!



> FSMJW EEYIV CVUXX FOISE HATAR ARVKT WKLVZ NGTVJ SYDNF BIUAL WPCFF LROSZ WGXFM EFRBM XYTDW FWPLZ JLQBI ATCID STSKH VKAAG VLNJL QJVDY SILFG QWGZZ MMFEA OIUOI JRMXL AIDRN QTRUT CXVFL MAEEY VKMSR ZWMES FRKIM FFALX VYSXZ MPXSJ MFRSG HZUGJ ROLBU KOCAX BHVUG WVBMA LWGQP WLWFM QQHBW XYXMY VFZMR TEDES TWFJM FAGKX YVJDB BWHUE ZRNKB ARSOA AHAXZ XDSRL LSWWI LRFLU BVSVV XXICH GGYRG HYXTS EFUVV LDWCZ TRBTA BZQVL VLOKJ VBVOF DOPWL QPUIJ ISLTU HRWYD LQVDM LURXE PKTHQ GRSLB RVRFL MHRPH UALJV LPAAU MAPWN FRWSI NOMUU VTLAV MJRDE IUMVG ZHAXY ISWBF LXFDE JQGPA PGTWN ETHMH RUVBZ ZPZQG ZWTJF YILNG MXVJF VBKXM TFGDV AXLPR EDVJB XVKFQ GVBLD MZRBX IVFTM KRARW HCPFR SFWSL GGZSF HYIRS ENXRK HRVIW OBKMF FLAGZ RPRLB PXILV EINVX HKOEL GVXIV PQWFL AIJLE LBTYK VWFOA VWIJV PSGWH GPXQJ RJIVV ZIFGG BMZIX TGGFC ZTEYV BTMPS ZOFLA IILKA BBHYI IMIGQ TRUKI DVUAM ALQFR ETVXH VWGHA XZYZW FXHVV CIJIW LBIPQ WGKUV FHHVV GDCBW FGXAK XZUKL USAHY MLCAS GHUPJ XHGBG NEBSE HXXLH PKCZX GKSGF GZXVV MSJJW MAFSG FYWTZ VTCKV GMXYM IWYDI YKZSE RHKNZ XUBCJ XGVKM FTBTO PKMHB JLXYL VWFBH PHRPT EGLXR YITNB BLOIP OAVLE ZSMFT COXYE OOYEL IRDIE NMUXD ERHRV MSRSE FQGNK WEEGV FZMED SFQSK LHRPW ATXEL ACWIS KRYIS WBFAM KOIJG CWBZG AJRJX HFUXZ RVTUP XMPYW ZPFII AGGIK VHGQG AHRJH RVSST MBVQG ZSRFV DMLUC NMLBM ACDXE JALWC VGHTI ZOBXM LVOIS ISGEF FARVW LYEKS MOHXW SCMFR AGXYV WWHBW BZGAJ RJXHJ VPAGI WXZAT OGETC EVXTR SQILG FSQAG ETVYF GFRHM IFSEF TPCPK ZGWFT FXTSE WWMJJ SNRFM ALAAB QJHYJ WSORF PAPGT OGLKE TAWLU SGXLH XSNFW QRFVW TIETA IMHUG NWRUH URZXL AMMZB TLIIC ELVCG LALMH EWJYZ YIGAZ RMOME JBQTK VASJR BWXYX TSVJL IVTMF TSVVL RFFVU BXZLW UBBLB ZXQBG XHVVC IJVGA TSPEO GATXV TCSER XGAGG FVGLM KFAAG VMALW UUULH JRWEJ GCYMO IICED WRVCI JOSYH YIHWF AMIUH RVZOR MYIMR NDTRU UINRF UXMSD SVEIV ZUXWQ PRMOI RCBLH JDHRL USLXH VQALW GXZJI ERBML HRPHU WREIL WMSTB VPIZH GGVSE XYWEO EEMIM FBXWE ENIJB FWXHX TOAVM SZUHM PSFXA SOCZE XRTLX ZVGET ISDWB MLZFF EYRKB MOXTS WGRET OMDQT XXSWI VRFAI VTFSE YLBUE XWGLE ISVEL JWMAO MEVBD BHRFQ SGSLH UEZSK HXHZA MGACY WPWOC IWKCL WLAFB TMPZQ FVNXV SBXKH DIHZM ZUNDE XYLWW PCGCL GFIEW LXFII XNZLX FSGQN FGSKJ SFGSL MALQW AWLXZ TETYS UXUIR WGOAM TOMKU OEEJS ZTRJH RRSPE NBDBU HFCGZ XPRZX YRBXK HXUCA TRHZZ GGISK BUKMD NKLEX LRWNF MALTA ZRLHX YVWWP CNGAV USFLH VVHGZ JVBVO EFDEW LIEAW GZOGR TSZHU KTVVY IIHWL BAIAF OQTWT LVLNW GBUKF VRKXG ILXGS HAXTE SBRLP LZJLA SOMTS PBCFK BFCLG SACGE FFQSS XXGKL HTLOG NUHQF GSDME NWMPV TLTMZ S





In [79]:
ciphertext = "FSMJW EEYIV CVUXX FOISE HATAR ARVKT WKLVZ NGTVJ SYDNF BIUAL WPCFF LROSZ WGXFM EFRBM XYTDW FWPLZ JLQBI ATCID STSKH VKAAG VLNJL QJVDY SILFG QWGZZ MMFEA OIUOI JRMXL AIDRN QTRUT CXVFL MAEEY VKMSR ZWMES FRKIM FFALX VYSXZ MPXSJ MFRSG HZUGJ ROLBU KOCAX BHVUG WVBMA LWGQP WLWFM QQHBW XYXMY VFZMR TEDES TWFJM FAGKX YVJDB BWHUE ZRNKB ARSOA AHAXZ XDSRL LSWWI LRFLU BVSVV XXICH GGYRG HYXTS EFUVV LDWCZ TRBTA BZQVL VLOKJ VBVOF DOPWL QPUIJ ISLTU HRWYD LQVDM LURXE PKTHQ GRSLB RVRFL MHRPH UALJV LPAAU MAPWN FRWSI NOMUU VTLAV MJRDE IUMVG ZHAXY ISWBF LXFDE JQGPA PGTWN ETHMH RUVBZ ZPZQG ZWTJF YILNG MXVJF VBKXM TFGDV AXLPR EDVJB XVKFQ GVBLD MZRBX IVFTM KRARW HCPFR SFWSL GGZSF HYIRS ENXRK HRVIW OBKMF FLAGZ RPRLB PXILV EINVX HKOEL GVXIV PQWFL AIJLE LBTYK VWFOA VWIJV PSGWH GPXQJ RJIVV ZIFGG BMZIX TGGFC ZTEYV BTMPS ZOFLA IILKA BBHYI IMIGQ TRUKI DVUAM ALQFR ETVXH VWGHA XZYZW FXHVV CIJIW LBIPQ WGKUV FHHVV GDCBW FGXAK XZUKL USAHY MLCAS GHUPJ XHGBG NEBSE HXXLH PKCZX GKSGF GZXVV MSJJW MAFSG FYWTZ VTCKV GMXYM IWYDI YKZSE RHKNZ XUBCJ XGVKM FTBTO PKMHB JLXYL VWFBH PHRPT EGLXR YITNB BLOIP OAVLE ZSMFT COXYE OOYEL IRDIE NMUXD ERHRV MSRSE FQGNK WEEGV FZMED SFQSK LHRPW ATXEL ACWIS KRYIS WBFAM KOIJG CWBZG AJRJX HFUXZ RVTUP XMPYW ZPFII AGGIK VHGQG AHRJH RVSST MBVQG ZSRFV DMLUC NMLBM ACDXE JALWC VGHTI ZOBXM LVOIS ISGEF FARVW LYEKS MOHXW SCMFR AGXYV WWHBW BZGAJ RJXHJ VPAGI WXZAT OGETC EVXTR SQILG FSQAG ETVYF GFRHM IFSEF TPCPK ZGWFT FXTSE WWMJJ SNRFM ALAAB QJHYJ WSORF PAPGT OGLKE TAWLU SGXLH XSNFW QRFVW TIETA IMHUG NWRUH URZXL AMMZB TLIIC ELVCG LALMH EWJYZ YIGAZ RMOME JBQTK VASJR BWXYX TSVJL IVTMF TSVVL RFFVU BXZLW UBBLB ZXQBG XHVVC IJVGA TSPEO GATXV TCSER XGAGG FVGLM KFAAG VMALW UUULH JRWEJ GCYMO IICED WRVCI JOSYH YIHWF AMIUH RVZOR MYIMR NDTRU UINRF UXMSD SVEIV ZUXWQ PRMOI RCBLH JDHRL USLXH VQALW GXZJI ERBML HRPHU WREIL WMSTB VPIZH GGVSE XYWEO EEMIM FBXWE ENIJB FWXHX TOAVM SZUHM PSFXA SOCZE XRTLX ZVGET ISDWB MLZFF EYRKB MOXTS WGRET OMDQT XXSWI VRFAI VTFSE YLBUE XWGLE ISVEL JWMAO MEVBD BHRFQ SGSLH UEZSK HXHZA MGACY WPWOC IWKCL WLAFB TMPZQ FVNXV SBXKH DIHZM ZUNDE XYLWW PCGCL GFIEW LXFII XNZLX FSGQN FGSKJ SFGSL MALQW AWLXZ TETYS UXUIR WGOAM TOMKU OEEJS ZTRJH RRSPE NBDBU HFCGZ XPRZX YRBXK HXUCA TRHZZ GGISK BUKMD NKLEX LRWNF MALTA ZRLHX YVWWP CNGAV USFLH VVHGZ JVBVO EFDEW LIEAW GZOGR TSZHU KTVVY IIHWL BAIAF OQTWT LVLNW GBUKF VRKXG ILXGS HAXTE SBRLP LZJLA SOMTS PBCFK BFCLG SACGE FFQSS XXGKL HTLOG NUHQF GSDME NWMPV TLTMZ S"
crack_vigenere(ciphertext)

Key length: 16
Candidate key: HESNOTTHEMONSTER
Plaintext: YOUWILLREJOICETOHEARTHATNODISASTERHASACCOMPANIEDTHECOMMENCEMENTOFANENTERPRISEWHICHYOUHAVEREGARDEDWITHSUCHEVILFOREBODINGSIARRIVEDHEREYESTERDAYANDMYFIRSTTASKISTOASSUREMYDEARSISTEROFMYWELFAREANDINCREASINGCONFIDENCEINTHESUCCESSOFMYUNDERTAKINGIAMALREADYFARNORTHOFLONDONANDASIWALKINTHESTREETSOFPETERSBURGHIFEELACOLDNORTHERNBREEZEPLAYUPONMYCHEEKSWHICHBRACESMYNERVESANDFILLSMEWITHDELIGHTDOYOUUNDERSTANDTHISFEELINGTHISBREEZEWHICHHASTRAVELLEDFROMTHEREGIONSTOWARDSWHICHIAMADVANCINGGIVESMEAFORETASTEOFTHOSEICYCLIMESINSPIRITEDBYTHISWINDOFPROMISEMYDAYDREAMSBECOMEMOREFERVENTANDVIVIDITRYINVAINTOBEPERSUADEDTHATTHEPOLEISTHESEATOFFROSTANDDESOLATIONITEVERPRESENTSITSELFTOMYIMAGINATIONASTHEREGIONOFBEAUTYANDDELIGHTTHEREMARGARETTHESUNISFOREVERVISIBLEITSBROADDISKJUSTSKIRTINGTHEHORIZONANDDIFFUSINGAPERPETUALSPLENDOURTHEREFORWITHYOURLEAVEMYSISTERIWILLPUTSOMETRUSTINPRECEDINGNAVIGATORSTHERESNOWANDFROSTAREBANISHEDANDSAILINGOVERACALMSEAWEMAYBEWAFTEDTOALANDSURPASSING